# Problema del Commesso Viaggiatore (TSP: Traveling Salesman problem)
È il seguente: Il commesso viaggiatore deve visitare ogni città del suo territorio esattamente una volta e poi tornare al
punto di partenza percorrendo la distanza più breve.

Possiamo rappresentare il problema avvalendoci di un grafo completo pesato, in cui i nodi sono le città e gli archi i collegamenti pesati,
che rappresentano la distanza tra le due città collegate.

Il problema è così definito:
   - Stati: un qualsiasi percorso che includa una sola volta tutte le n città. Possiamo rappresentare uno stato con una lista di città lunga n.
   - Funzione successore: un qualsiasi scambio di due città.
   - Funzione di valutazione: calcolo lunghezza percorso.
   - Stato iniziale: una lista di città lunga n


In [1]:
import copy

# connessioni tra le citta con relative distanze
connections = {}
connections['A'] = [ ['B', 4], ['C', 2], ['D', 1], ['E', 2]]
connections['B'] = [ ['A', 4], ['C', 1], ['D', 2], ['E', 3]]
connections['C'] = [ ['A', 2], ['B', 1], ['D', 3], ['E', 2]]
connections['D'] = [ ['A', 1], ['B', 2], ['C', 3], ['E', 2]]
connections['E'] = [ ['A', 2], ['B', 3], ['C', 2], ['D', 2]]

## FUNZIONE DI VALUTAZIONE DELLO STATO
Tale funzione calcola la lunghezza del percorso relativo ad uno stato, rappresentato mediante una lista ordinata di città

In [2]:
def eval_stato(stato):
    lunghezza_percorso = 0
    for i in range(len(stato)):
        l = connections[stato[i]]
        if i < (len(stato)-1):
            next_nodo = stato[i+1]
        else:
            next_nodo = stato[0]
        for (successore, distanza) in l:
            if successore == next_nodo:
                lunghezza_percorso += distanza
                break
    return lunghezza_percorso

## FUNZIONE CHE GENERA IL NEIGHBORHOOD
Tale funzione genera gli stati del vicinato, ordinandoli in base alla loro valutazione (lunghezza percorso)

In [3]:
def genera_successori(stato):
    '''
    GENERA LA LISTA ORDINATA DI SUCCESSORI DI UNO STATO
    :param stato:
    :return:
    '''
    lista = []
    t = len(stato)

    for i in range(0, t-1):
        for j in range(i+1, t):
            buffer = copy.deepcopy(stato)
            temp = buffer[i]
            buffer[i] = buffer[j]
            buffer[j] = temp
            eval_next = eval_stato(buffer)
            lista.append( (buffer, eval_next, (stato[i], stato[j])))

    lista.sort(key=lambda x:x[1])   #ordiniamo i successori in base alla loro valutazione
    return lista

La funzione precedente, per ogni coppia di indici i e j dello
stato, scambia i valori degli elementi corrispondenti. Ad
esempio:    statox = [A, B, C, D, E] DIVENTA stato_successore=[A, D, C, B, E] con cambia a i=B e j=D

## FUNZIONE TABU TEST
Tale funzione controlla se una mossa è ASSENTE nella Tabu List

In [4]:
def tabu_test(sequenza, tabu_list): #è True se una mossa NON è presente
    a, b = sequenza[2]
    if ( (a,b) in tabu_list or (b,a) in tabu_list):
        assente = False
    else:
        assente = True
    return assente

## ALGORITMO

In [5]:
def tabu_search(stato_iniziale, tabu_tenure, connections):
    # impostazione stato iniziale
    current = stato_iniziale
    current_eval = eval_stato(current)

    #inizializziamo il best
    best = stato_iniziale
    eval_best = current_eval

    tabu_list = {}
    neighbours = []
    count = 0

    print('STATO INIZIALE(', str(count), '):')
    print(current)
    print('EVAL STATO INIZIALE(', str(count), '):')
    print(current_eval)
    print('----------------------')

    #while not criterio_di_terminazione
    while count < 10:

        lista_successori = genera_successori(current)   #genera i successori (stato, eval_stato, mossa)

        neighbours = list(filter(lambda n: tabu_test(n, tabu_list), lista_successori))  #selezione successori non tabu

        next_state = neighbours[0][0]   #selezione del migliore dei successori
        eval_next_state = neighbours[0][1]
        delta = eval_best - eval_next_state

        if delta>0:
            best=next_state
            eval_best = eval_next_state

        current = next_state
        current_eval = eval_next_state

        print('STATO CORRENTE(', str(count+1), '):')
        print(current)
        print('EVAL STATO CORRENTE(', str(count+1), '):')
        print(current_eval)

        #decremento del tabu_tenure
        for mossa in tabu_list:
            tabu_list[mossa] = tabu_list[mossa] - 1

        #eliminazione elementi con tenure uguale a zero
        tabu_list = {k: tabu_list[k] for k in tabu_list if tabu_list[k] != 0}

        #inserimento della mossa di next in tabu_list
        mossa_next = neighbours[0][2]
        print('MOSSA(', str(count+1), '):')
        print(mossa_next)
        tabu_list[mossa_next] = tabu_tenure

        count += 1

        print('----------------------')

    return (best + [best[0]], eval_best)

In [6]:
stato = ['A', 'B', 'C', 'D', 'E']

In [7]:
tabu_search(stato, 3, connections)

STATO INIZIALE( 0 ):
['A', 'B', 'C', 'D', 'E']
EVAL STATO INIZIALE( 0 ):
12
----------------------
STATO CORRENTE( 1 ):
['D', 'B', 'C', 'A', 'E']
EVAL STATO CORRENTE( 1 ):
9
MOSSA( 1 ):
('A', 'D')
----------------------
STATO CORRENTE( 2 ):
['C', 'B', 'D', 'A', 'E']
EVAL STATO CORRENTE( 2 ):
8
MOSSA( 2 ):
('D', 'C')
----------------------
STATO CORRENTE( 3 ):
['C', 'B', 'D', 'E', 'A']
EVAL STATO CORRENTE( 3 ):
9
MOSSA( 3 ):
('A', 'E')
----------------------
STATO CORRENTE( 4 ):
['C', 'A', 'D', 'E', 'B']
EVAL STATO CORRENTE( 4 ):
9
MOSSA( 4 ):
('B', 'A')
----------------------
STATO CORRENTE( 5 ):
['C', 'A', 'E', 'D', 'B']
EVAL STATO CORRENTE( 5 ):
9
MOSSA( 5 ):
('D', 'E')
----------------------
STATO CORRENTE( 6 ):
['D', 'A', 'E', 'C', 'B']
EVAL STATO CORRENTE( 6 ):
8
MOSSA( 6 ):
('C', 'D')
----------------------
STATO CORRENTE( 7 ):
['D', 'E', 'A', 'C', 'B']
EVAL STATO CORRENTE( 7 ):
9
MOSSA( 7 ):
('A', 'E')
----------------------
STATO CORRENTE( 8 ):
['D', 'E', 'B', 'C', 'A']
EVAL ST

(['C', 'B', 'D', 'A', 'E', 'C'], 8)